In [1]:
from __future__ import print_function, division
import argparse
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import torch.utils.data
import time
from datasets import __datasets__
from datasets.data_io import get_transform
import gc
import skimage
import skimage.io
import skimage.transform
from models.bgnet import BGNet
from models.bgnet_plus import BGNet_Plus
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import math
import copy
import sys
from PIL import Image

In [2]:
from torch.utils.tensorboard import SummaryWriter 
writer = SummaryWriter('./log')

2024-01-11 11:46:07.356049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-11 11:46:07.497574: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-11 11:46:08.166770: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /disk2/users/M22_zhaoqinghao/.local/lib/python3.8/site-packages/cv2/../../lib64:/opt/ro

In [3]:
class Args:
  model = 'bgnet_plus'
  dataset = 'dsec_png'
  datapath = '/home/zhaoqinghao/dataset/DSEC/output'
  savepath = '/disk2/users/M22_zhaoqinghao/BGNet/output/'
  trainlist = '/home/zhaoqinghao/DSEC/train.txt'
  testlist = '/home/zhaoqinghao/DSEC/test.txt'
  loadmodel = None
  savemodel = './'
  epochs = 233
  no_cuda = False
  seed = 1

args = Args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [4]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

datapath = args.datapath
StereoDataset = __datasets__[args.dataset]

dsec_train = args.trainlist
dsec_train_dataset = StereoDataset(datapath, dsec_train, True)
TrainImgLoader = DataLoader(dsec_train_dataset, batch_size= 52, shuffle=True, num_workers=32, drop_last=False)

dsec_test = args.testlist
dsec_test_dataset = StereoDataset(datapath, dsec_test, False)
TestImgLoader = DataLoader(dsec_test_dataset, batch_size= 8, shuffle=False, num_workers=4, drop_last=False)

if args.model == 'bgnet':
    model = BGNet().cuda()
elif args.model == 'bgnet_plus':
    model = BGNet_Plus().cuda()

if args.cuda:
    model.cuda()

if args.loadmodel is not None:
    state_dict = torch.load(args.loadmodel, map_location=torch.device('cuda' if args.cuda else 'cpu'))
    model.load_state_dict(state_dict.get('state_dict', {}), strict=False)
    print('load model')
    print(args.loadmodel)
print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-4)

Number of model parameters: 5315811


In [ ]:
def train(imgL, imgR, disp_L):
    model.train()
    
    if args.cuda:
        imgL, imgR, disp_true = imgL.cuda(), imgR.cuda(), disp_L.cuda()

    optimizer.zero_grad()

    output, _ = model(imgL, imgR)

    disp_true = torch.squeeze(disp_true, 0)
    disp_true = torch.squeeze(disp_true, 1)

    mask = (disp_true > 0)
    mask.detach_()
    
    loss = F.smooth_l1_loss(output[mask], disp_true[mask], size_average=True)

    loss.backward()
    optimizer.step()

    return loss.data



In [19]:
def test(imgL,imgR,disp_true):
    model.eval()
    with torch.no_grad():
        pred,_ = model(imgL.cuda(), imgR.cuda())
    disp_true = torch.squeeze(disp_true, 1)
    pred_disp = pred.data.cpu()
    mask = disp_true > 0
    disp_diff = torch.abs(disp_true[mask] - pred_disp[mask])
    err_mask = disp_diff > 3    # 3 pixel error
    return torch.mean(err_mask.float())

# def Thres_metric(D_est, D_gt, mask, thres):
#     assert isinstance(thres, (int, float))
#     D_est, D_gt = D_est[mask], D_gt[mask]
#     E = torch.abs(D_gt - D_est)
#     err_mask = E > thres
#     return torch.mean(err_mask.float())


In [20]:
total_test_loss = 0
for batch_idx, sample in enumerate(TestImgLoader):
    imgL, imgR, disp_L = sample['left'], sample['right'], sample['disparity']
    test_loss = test(imgL, imgR, disp_L)
    writer.add_scalar('Test Loss', test_loss, batch_idx)
    print('Iter %d 3-px Accuracy in val = %.3f' %(batch_idx, test_loss*100))
    total_test_loss += test_loss

# print('epoch %d total 3-px Accuracy in val = %.3f' %(epoch, total_test_loss/len(TestImgLoader)*100))

# if total_test_loss/len(TestImgLoader)*100 > max_acc:
#     max_acc = total_test_loss/len(TestImgLoader)*100
#     max_epo = epoch

# print('MAX epoch %d total test Accuracy = %.3f' %(max_epo, max_acc))

# print('full finetune time = %.2f HR' %((time.time() - start_full_time)/3600))
# print(max_epo)
# print(max_acc)

Iter 0 3-px Accuracy in val = 1.656
Iter 1 3-px Accuracy in val = 1.346
Iter 2 3-px Accuracy in val = 1.629
Iter 3 3-px Accuracy in val = 1.816
Iter 4 3-px Accuracy in val = 1.296
Iter 5 3-px Accuracy in val = 1.701
Iter 6 3-px Accuracy in val = 1.858
Iter 7 3-px Accuracy in val = 1.422
Iter 8 3-px Accuracy in val = 1.152
Iter 9 3-px Accuracy in val = 1.589
Iter 10 3-px Accuracy in val = 2.173
Iter 11 3-px Accuracy in val = 2.388
Iter 12 3-px Accuracy in val = 2.038
Iter 13 3-px Accuracy in val = 2.241
Iter 14 3-px Accuracy in val = 2.028
Iter 15 3-px Accuracy in val = 1.125
Iter 16 3-px Accuracy in val = 1.488
Iter 17 3-px Accuracy in val = 1.477
Iter 18 3-px Accuracy in val = 1.884
Iter 19 3-px Accuracy in val = 1.868
Iter 20 3-px Accuracy in val = 1.292
Iter 21 3-px Accuracy in val = 1.606
Iter 22 3-px Accuracy in val = 2.010
Iter 23 3-px Accuracy in val = 1.717
Iter 24 3-px Accuracy in val = 1.272
Iter 25 3-px Accuracy in val = 1.440
Iter 26 3-px Accuracy in val = 1.028
Iter 27 3-p

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    if epoch <= 200:
       lr = 0.001
    else:
       lr = 0.0001
    print('learning rate = %.6f' %(lr))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr



In [ ]:
%load_ext tensorboard
%tensorboard --logdir log

In [ ]:
max_acc = 0
max_epo = 0
start_full_time = time.time()

for epoch in range(1, args.epochs+1):
    print('This is %d-th epoch' %(epoch))
    total_train_loss = 0
    adjust_learning_rate(optimizer, epoch)
    print('epoch %d' %(epoch))
    for batch_idx, sample in enumerate(TrainImgLoader):
        start_time = time.time()
        imgL, imgR, disp_L = sample['left'], sample['right'], sample['disparity']
        loss = train(imgL.cuda(), imgR.cuda(), disp_L.cuda())
        
        # Record batch_idx and loss to TensorBoard
        writer.add_scalar('Train Loss', loss, epoch * len(TrainImgLoader)+batch_idx)
        
        # print('Iter %d training loss = %.3f , time = %.2f' %(batch_idx, loss, time.time() - start_time))
        total_train_loss += loss
    print('epoch %d total training loss = %.3f' %(epoch, total_train_loss/len(TrainImgLoader)))
    
    # Record epoch and total_train_loss to TensorBoard
    writer.add_scalar('Total Train Loss', total_train_loss/len(TrainImgLoader), epoch)
    
    if epoch % 10 == 0:
        savefilename = args.savemodel+'checkpoint_'+str(epoch)+'.pth'
        torch.save(model.state_dict(), savefilename)

In [ ]:
epoch = 0
max_acc = 0